# CSMCC16 Flight Coursework

### Import packages

In [27]:
import numpy as np
import pandas as pd
import datetime
import re

### Import data

In [200]:
pass_data = pd.read_csv('AComp_Passenger_data.csv',header=None)   #comment out as necessary
#pass_data = pd.read_csv('AComp_Passenger_data_no_error.csv',header=None)   #comment out as necessary
pass_datet = pd.read_csv('AComp_Passenger_data_no_error_DateTime.csv',header=None)
airport_lat_lon = pd.read_csv('Top30_airports_LatLong.csv',header=None)

In [31]:
pass_data

,0,1,2,3,4,5
0,UES9151GS5,SQU6245R,DEN,FRA,1420564460,1049
1,UES9151GS5,XXQ4064B,JFK,FRA,1420563917,802
2,EZC9678QI6,SOH3431A,ORD,MIA,1420563649,250
3,ONL0812DH1,SOH3431A,ORD,MIA,1420563649,250
4,CYJ0225CH1,PME8178S,DEN,PEK,1420564409,1322
...,...,...,...,...,...,...
505,LLZ3798PE3,EWH6301Y,CAN,DFW,1420564967,1683
506,KKP5277HZ7,KJR6646J,IAH,BKK,1420565203,1928
507,JJM4724RF7,XXQ4064B,JFK,FRA,1420563917,802
508,SJD8775RZ4,WSK1289Z,CLT,vEN,1420563542,278


In [201]:
# Create as dataframes
pass_data_df = pd.DataFrame(pass_data)
pass_datet_df = pd.DataFrame(pass_datet)
airport_df = pd.DataFrame(airport_lat_lon)

In [204]:
# Add Column Names
pass_data_df.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min']
airport_df.columns = ['Airport_name'
                        ,'Airport_IATA_code'
                        ,'Latitude'
                        ,'Longitude']
pass_datet_df.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Departure_date_time'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min'
                        ,'Arrive_date_time'
]

### Error Correction

In [34]:
#Remove values where passenger id and flight id are missing
pass_data_df=pass_data_df.dropna(subset=['Passenger_id', 'Flight_id'])

In [35]:
pass_data_df['count']='1'

In [36]:
pass_data_df

,Passenger_id,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time_epoch,Flight_time_min,count
0,UES9151GS5,SQU6245R,DEN,FRA,1420564460,1049,1
1,UES9151GS5,XXQ4064B,JFK,FRA,1420563917,802,1
2,EZC9678QI6,SOH3431A,ORD,MIA,1420563649,250,1
3,ONL0812DH1,SOH3431A,ORD,MIA,1420563649,250,1
4,CYJ0225CH1,PME8178S,DEN,PEK,1420564409,1322,1
...,...,...,...,...,...,...,...
504,BWI0520BG6,BER7172M,KUL,LAS,1420565167,1848,1
505,LLZ3798PE3,EWH6301Y,CAN,DFW,1420564967,1683,1
506,KKP5277HZ7,KJR6646J,IAH,BKK,1420565203,1928,1
507,JJM4724RF7,XXQ4064B,JFK,FRA,1420563917,802,1


In [71]:
#correct formats
format_IATA = r"[A-Z][A-Z][A-Z]"
format_flight_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"

#add column indicating if correct
pass_data_df['From_airport_format'] =pass_data_df['From_airport_IATA'].str.contains(format_IATA)
pass_data_df['Dest_airport_format'] =pass_data_df['Dest_airport_IATA'].str.contains(format_IATA)
pass_data_df['Pass_id_format'] =pass_data_df['Passenger_id'].str.contains(format_pass_id)
pass_data_df['Flight_id_format'] =pass_data_df['Flight_id'].str.contains(format_flight_id)

In [ ]:
#add column indicating if all flight regex correct
pass_data_df['Flight_info_correct']=pass_data_df.all(axis='columns')

In [86]:
#select only those whose regex is correct
flight_info_true = pass_data_df.loc[pass_data_df['Flight_info_correct'] == True]

In [197]:
flight_info_true

,Passenger_id,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time_epoch,Flight_time_min,count,From_airport_format,From_airport_IATA_format,Dest_airport_IATA_format,Dest_airport_format,Pass_id_format,Flight_id_format,Flight_info_correct
0,UES9151GS5,SQU6245R,DEN,FRA,1420564460,1049,1,True,True,True,True,True,True,True
1,UES9151GS5,XXQ4064B,JFK,FRA,1420563917,802,1,True,True,True,True,True,True,True
2,EZC9678QI6,SOH3431A,ORD,MIA,1420563649,250,1,True,True,True,True,True,True,True
3,ONL0812DH1,SOH3431A,ORD,MIA,1420563649,250,1,True,True,True,True,True,True,True
4,CYJ0225CH1,PME8178S,DEN,PEK,1420564409,1322,1,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,WTC9125IE5,GMO5938W,LHR,PEK,1420564317,1057,1,True,True,True,True,True,True,True
504,BWI0520BG6,BER7172M,KUL,LAS,1420565167,1848,1,True,True,True,True,True,True,True
505,LLZ3798PE3,EWH6301Y,CAN,DFW,1420564967,1683,1,True,True,True,True,True,True,True
506,KKP5277HZ7,KJR6646J,IAH,BKK,1420565203,1928,1,True,True,True,True,True,True,True


In [99]:
#check against airport list
airport_lat_lon['Airport_IATA_code']
pass_data_df['From_airport_IATA']
pass_data_df['Dest_airport_IATA']

0      FRA
1      FRA
2      MIA
3      MIA
4      PEK
      ... 
504    LAS
505    DFW
506    BKK
507    FRA
508    vEN
Name: Dest_airport_IATA, Length: 500, dtype: object

In [243]:
#Take True airport values, change column names to enable merge

from_values = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
from_values.columns = ['From_airport_IATA']
#add column to know if exists
from_values['From_airport_exists']=True

dest_values = pd.DataFrame(airport_lat_lon.Airport_IATA_code.unique())
#rename columns to allow merge
dest_values.columns = ['Dest_airport_IATA']
#add column to know if exists
dest_values['Dest_airport_exists']=True

In [233]:
merge_with_dest.to_csv(r'merge_with_dest.csv')

In [244]:
#merge with from
merge_with_from=flight_info_true.merge(from_values
                     , how='outer', on=['From_airport_IATA'])
#merge with dest
merge_with_dest=merge_with_from.merge(dest_values
                     , how='outer', on=['Dest_airport_IATA'])
merge_with_dest

,Passenger_id,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time_epoch,Flight_time_min,count,From_airport_format,From_airport_IATA_format,Dest_airport_IATA_format,Dest_airport_format,Pass_id_format,Flight_id_format,Flight_info_correct,From_airport_exists,Dest_airport_exists
0,UES9151GS5,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
1,JBE2302VO4,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
2,HCA3158QA6,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
3,XFG5747ZT9,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
4,PIT2755XC1,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,NaN,NaN,NaN,IAH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
489,NaN,NaN,NaN,CLT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
490,NaN,NaN,NaN,MUC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
491,NaN,NaN,NaN,KUL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [242]:
merge_with_dest

,Passenger_id,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time_epoch,Flight_time_min,count,From_airport_format,From_airport_IATA_format,Dest_airport_IATA_format,Dest_airport_format,Pass_id_format,Flight_id_format,Flight_info_correct,from_airport_exists,Dest_airport_exists
0,UES9151GS5,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
1,JBE2302VO4,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
2,HCA3158QA6,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
3,XFG5747ZT9,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
4,PIT2755XC1,SQU6245R,DEN,FRA,1.420564e+09,1049.0,1,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,NaN,NaN,NaN,IAH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
489,NaN,NaN,NaN,CLT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
490,NaN,NaN,NaN,MUC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
491,NaN,NaN,NaN,KUL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [245]:
#drop NaNs
merge_with_dest=merge_with_dest.dropna(subset=['From_airport_exists', 'Dest_airport_exists'])

In [247]:
flight_only_data = merge_with_dest[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time_epoch'
                      ,'Flight_time_min'
#                       ,'count'
                        ]]

In [249]:
#airport_only_grp = 
flight_only_data.groupby(['Flight_id'
                          ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time_epoch'
                      ,'Flight_time_min'           
                         ]).count()
#airport_only_grp

,,,,
Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time_epoch,Flight_time_min
ATT7791R,AMS,DEN,1.420564e+09,1001.0
BER7172M,KUL,LAS,1.420565e+09,1848.0
DAU2617A,CGK,SFO,1.420565e+09,1811.0
DKZ3042O,MIA,SFO,1.420564e+09,538.0
EWH6301Y,CAN,DFW,1.420565e+09,1683.0
FYL5866L,ATL,HKG,1.420565e+09,1751.0
GMO5938W,LHR,PEK,1.420564e+09,1057.0
HUR0974O,DEN,PVG,1.420565e+09,1398.0
HZT2506M,IAH,AMS,1.420564e+09,1044.0


In [159]:
airport_only_grp = flight_only_true_data.groupby(['From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time_epoch'
                      ,'Flight_time_min'           
                         ]).count()
airport_only_grp

Flight_id
From_airport_IATA Dest_airport_IATA Depart_time_epoch Flight_time_min           
AMS               DEN               1420564394        1001                    13
ATL               HKG               1420565140        1751                    19
                  LHR               1420564038        877                     15
BKK               MIA               1420565330        2027                    16
CAN               DFW               1420564967        1683                     9
                                    1420564983        1683                    26
CDG               LAS               1420564498        1133                    19
CGK               DXB               1420563958        849                     14
                  SFO               1420564986        1811                    12
CLT               DEN               1420563542        278                     20
DEN               FRA               1420564460        1049                    19
                  PEK               1420564409        1322                    18
                  PVG               1420564525        1398                     7
DFW               PEK               1420564869        1452                    10
FCO               LAS               1420564698        1276                    14
HND               CAN               1420563569        374                     13
IAH               AMS               1420564324        1044                    13
                  BKK               1420565203        1928                    23
JFK               FRA               1420563917        802                     23
                  FSA               1420563917        802                      1
KUL               LAS               1420565167        1848                    16
                  PEK               1420563856        572                     15
LAS               SIN               1420565203        1843                    14
LHR               PEK               1420564317        1057                    24
MAD               FRA               1420563408        184                     12
MIA               SFO               1420563927        538                     11
MUC               MAD               1420563539        194                     13
ORD               DXB               1420564716        1510                    15
                  MIA               1420563649        250                     17
PEK               LAX               1420564414        1302                    11
PVG               FCO               1420564561        1189                    20
UGK               DXB               1420563958        849                      1

In [155]:
airport_only_grp.reset_index(inplace=True)

In [157]:
#data for merging
airport_only_from = airport_only_grp[['From_airport_IATA']]
airport_only_dest = airport_only_grp[['Dest_airport_IATA']]
#rename columns to allow merge
airport_only_from.columns = ['Airport_IATA_code']
airport_only_dest.columns = ['Airport_IATA_code']

In [158]:
airport_only_from.merge(airport_lat_lon, how='outer', on=['Airport_IATA_code','Airport_IATA_code'])

,Airport_IATA_code,Airport_name,Latitude,Longitude
0,AMS,AMSTERDAM,52.308613,4.763889
1,ATL,ATLANTA,33.636719,-84.428067
2,ATL,ATLANTA,33.636719,-84.428067
3,BKK,BANGKOK,13.681108,100.747283
4,CAN,GUANGZHOU,23.392436,113.298786
5,CAN,GUANGZHOU,23.392436,113.298786
6,CDG,PARIS,49.012779,2.550000
7,CGK,JAKARTA,-6.125567,106.655897
8,CGK,JAKARTA,-6.125567,106.655897
9,CLT,CHARLOTTE,35.214000,-80.943139


In [105]:
#flight info
#flight_from = 
flight_only_true_data[['Flight_id','From_airport_IATA']]
#flight_dest = obj2_results[['Flight_id','Dest_airport_IATA']]

,Flight_id,From_airport_IATA
0,SQU6245R,DEN
1,XXQ4064B,JFK
2,SOH3431A,ORD
3,SOH3431A,ORD
4,PME8178S,DEN
...,...,...
503,GMO5938W,LHR
504,BER7172M,KUL
505,EWH6301Y,CAN
506,KJR6646J,IAH


In [103]:
#rename columns to allow merge
flight_only_grp.columns = ['Flight_id'
                        ,'Airport_IATA_code']
#flight_dest.columns = ['Flight_id'
 #                       ,'Airport_IATA_code']

In [104]:
flight_from.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])


,Flight_id,Airport_IATA_code,Airport_name,Latitude,Longitude
0,SQU6245R,DEN,DENVER,39.861656,-104.673178
1,PME8178S,DEN,DENVER,39.861656,-104.673178
2,HUR0974O,DEN,DENVER,39.861656,-104.673178
3,HUR0974O,DEN,DENVER,39.861656,-104.673178
4,SQU6245R,DEN,DENVER,39.861656,-104.673178
...,...,...,...,...,...
467,RPG3351U,HND,TOKYO,35.552258,139.779694
468,RPG3351U,HND,TOKYO,35.552258,139.779694
469,RPG3351U,HND,TOKYO,35.552258,139.779694
470,RPG3351U,HND,TOKYO,35.552258,139.779694


In [80]:
#export data
pass_data_df.to_csv(r'check format.csv')

In [72]:
pass_data_df

,Passenger_id,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time_epoch,Flight_time_min,count,From_airport_format,From_airport_IATA_format,Dest_airport_IATA_format,Dest_airport_format,Pass_id_format,Flight_id_format
0,UES9151GS5,SQU6245R,DEN,FRA,1420564460,1049,1,True,True,True,True,True,True
1,UES9151GS5,XXQ4064B,JFK,FRA,1420563917,802,1,True,True,True,True,True,True
2,EZC9678QI6,SOH3431A,ORD,MIA,1420563649,250,1,True,True,True,True,True,True
3,ONL0812DH1,SOH3431A,ORD,MIA,1420563649,250,1,True,True,True,True,True,True
4,CYJ0225CH1,PME8178S,DEN,PEK,1420564409,1322,1,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,BWI0520BG6,BER7172M,KUL,LAS,1420565167,1848,1,True,True,True,True,True,True
505,LLZ3798PE3,EWH6301Y,CAN,DFW,1420564967,1683,1,True,True,True,True,True,True
506,KKP5277HZ7,KJR6646J,IAH,BKK,1420565203,1928,1,True,True,True,True,True,True
507,JJM4724RF7,XXQ4064B,JFK,FRA,1420563917,802,1,True,True,True,True,True,True


In [74]:
#select data for objective 2
#obj2_data = 
flight_only_data = pass_data_df[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time_epoch'
                      ,'Flight_time_min'
                        ,'From_airport_format'  
                        ,'Dest_airport_format'
#                        ,'Pass_id_format'
                        ,'Flight_id_format'
                    ,'count'
                        ]]

In [77]:
#obj2_results = 
flight_only_data.groupby(['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time_epoch'
                      ,'Flight_time_min'
                        ,'From_airport_format'  
                        ,'Dest_airport_format'
                        ,'Pass_id_format'
                        ,'Flight_id_format'                         
                         ]).count()
#reset index as columns
#flight_only_data.reset_index(inplace=True)
##obj2_results

index  \
Flight_id From_airport_IATA Dest_airport_IATA Depart_time_epoch Flight_time_min From_airport_format Dest_airport_format Pass_id_format Flight_id_format          
4QU6245R  DEN               FRA               1420564460        1049            True                True                True           False                 1   
ATT7791R  A;S               DEN               1420564394        1001            False               True                True           True                  1   
          AM%               DEN               1420564394        1001            False               True                True           True                  1   
          AMS               DEN               1420564394        1001            True                True                True           True                 13   
BER7172M  KUL               LAS               1420565167        1848            True                True                True           True                 16   
                            [AS               1420565167        1848            True                False               True           True                  1   
DAU2617A  CGK               SFO               1420564986        1811            True                True                True           True                 12   
DKZ3042O  MIA               SFO               1420563927        538             True                True                True           True                 11   
EWH6301Y  CAN               DFW               1420564967        1683            True                True                True           True                  9   
          ~AN               DFW               1420564967        1683            False               True                True           True                  1   
FYL5866L  ATL               HKG               1420565140        1751            True                True                False          True                  1   
                                                                                                                        True           True                 19   
GMO5938W  LHR               PEK               1420564317        1057            True                True                True           True                 24   
GMO593[W  LHR               PEK               1420564317        1057            True                True                True           False                 1   
HUR0974O  DEN               PVG               1420564525        1398            True                True                True           True                  7   
HZT2506M  IAH               #MS               1420564324        1044            True                False               True           True                  1   
                            AMS               1420564324        1044            True                True                True           True                 13   
JVY9791G  PVG               FCO               1420564561        1189            True                True                True           True                 20   
KJR6646J  IAH               BKK               1420565203        1928            True                True                True           True                 23   
MBA8071P  KUL               PEK               1420563856        572             True                True                True           True                 15   
MBw8071P  KUL               PEK               1420563856        572             True                True                True           False                 1   
MOO1786A  MAD               FRA               1420563408        184             True                True                True           True                 12   
                            ]RA               1420563408        184             True                False               True           True                  1   
PME8178S  DEN               PEK               1420564409        1322            True                Tru

### Additional Columns and formats

In [ ]:
#Calculate epoch Arrival Time
pass_data_df['Arrive_epoch'] = pass_data_df['Depart_time_epoch']+pass_data_df['Flight_time_min']*60
#Convert epoch to datetime
pass_data_df['Depart_time_datetime'] = pd.to_datetime(pass_data_df['Depart_time_epoch'],unit='s')
pass_data_df['Arrive_time_datetime'] = pd.to_datetime(pass_data_df['Arrive_epoch'],unit='s')

In [ ]:
#And new datetime formats
pass_data_df['Depart_day_month']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
#pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
pass_data_df['Depart_time']=pass_data_df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
pass_data_df['Arrive_time']=pass_data_df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")


In [ ]:
# Check data
print(pass_data_df)
print(pass_datet_df)
print(airport_df)

## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [ ]:
# List of top 30 airports
obj_1_list = airport_df['Airport_IATA_code']
obj_1_list.columns = ['From_airport_IATA']
obj_1_list_df = pd.DataFrame(obj_1_list)

In [ ]:
#check data
obj_1_list_df

In [ ]:
# Count number of departure flights
from_df = pass_data_df['From_airport_IATA']
from_counts = from_df.value_counts()
from_counts_df = pd.DataFrame(from_counts)
# move index to column
from_counts_df['Airport_IATA_code'] =from_counts_df.index

In [ ]:
#Check data
print(from_df)
print(from_counts)
print(from_counts_df)

In [ ]:
# merge full list of airports with number of flights from each airport
obj1 = obj_1_list_df.merge(from_counts_df, how='outer', on=['Airport_IATA_code', 'Airport_IATA_code'])

In [ ]:
# clean up results
obj1=obj1.fillna(0)
obj1=obj1.sort_values(by=['From_airport_IATA','Airport_IATA_code'], ascending=[False,True])
obj1.set_index('Airport_IATA_code', inplace=True, drop=True)
obj1

## Objective 2
### Create a list of flights based on the Flight id, this output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

In [13]:
#select data for objective 2
obj2_data = pass_data_df[['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Arrive_time'
                      ,'Flight_time_min'
                      ,'Passenger_id']]

KeyError: "['Arrive_time', 'Depart_time', 'Depart_day_month'] not in index"

## Objective 3
### Calculate the number of passengers on each flight.

In [ ]:
obj2_results = obj2_data.groupby(['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Arrive_time'
                      ,'Flight_time_min']).count()
#reset index as columns
obj2_results.reset_index(inplace=True)
obj2_results

## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [ ]:
#flight info
flight_from = obj2_results[['Flight_id','From_airport_IATA']]
flight_dest = obj2_results[['Flight_id','Dest_airport_IATA']]

In [ ]:
#rename columns to allow merge
flight_from.columns = ['Flight_id'
                        ,'Airport_IATA_code']
flight_dest.columns = ['Flight_id'
                        ,'Airport_IATA_code']

In [ ]:
#merge from flight with latitude and longitude
from_lat_long = flight_from.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
from_lat_long.columns = ['Flight_id'
                        ,'From_Airport_IATA_code'
                        ,'From_Airport_Name'
                        ,'From_Lat'
                        ,'From_Lon'
                    ]

In [ ]:
#merge dest flight with latitude and longitude
dest_lat_lon = flight_dest.merge(airport_lat_lon, how='inner', on=['Airport_IATA_code','Airport_IATA_code'])
dest_lat_lon.columns = ['Flight_id'
                        ,'Dest_Airport_IATA_code'
                        ,'Dest_Airport_Name'
                        ,'Dest_Lat'
                        ,'Dest_Lon'
                    ]

In [ ]:
#merge from flight with latitude and longitude
from_dest_lat_lon = from_lat_long.merge(dest_lat_lon, how='outer', on=['Flight_id','Flight_id'])

In [ ]:
from_dest_lat_lon

In [ ]:
# Calculate nautical miles (a^2+b^2=c^2)
from_dest_lat_lon['Miles'] = np.sqrt((from_dest_lat_lon['Dest_Lat']-from_dest_lat_lon['From_Lat'])**2
+(from_dest_lat_lon['Dest_Lon']-from_dest_lat_lon['From_Lon'])**2)

In [ ]:
#export results
from_dest_lat_lon.to_csv(r'from_dest_lat_lon.csv')

In [ ]:
# Number of miles per flight 
miles_flight = from_dest_lat_lon[['Flight_id','Miles']]
miles_flight

In [ ]:
#Add flight miles to passenger data
obj4_data = obj2_data.merge(miles_flight, how='outer', on=['Flight_id', 'Flight_id'])
obj4_data

In [ ]:
#Number of miles per flight per passenger
pass_flights= pd.pivot_table(obj4_data, values='miles', index=['Passenger_id'],
               columns=['Flight_id']
              , aggfunc=np.sum
              )
pass_flights

In [ ]:
pass_flights.to_csv(r'pass_flights.csv')

In [ ]:
#Add up flight miles per passenger
Pass_miles = pass_flights.sum(axis = 1, skipna = True) 
Pass_miles_df = pd.DataFrame(Pass_miles)
Pass_miles_df.columns = ['Air_miles']
Pass_miles_df = Pass_miles_df.sort_values(by=['Air_miles'], ascending=False)
Pass_miles_df.to_csv(r'Pass_miles_df.csv')
Pass_miles_df

# old code, maybe useful

In [ ]:
pass_data_df.value_counts(['From_airport_IATA'])

In [ ]:
pass_data_df.groupby(['From_airport_IATA']).count()

In [125]:
flight_only_grp.isin(flight_only_grp['From_airport_IATA'])

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time_epoch,Flight_time_min
0,False,True,False,False,False
1,False,True,False,False,False
2,False,True,False,False,False
3,False,True,False,False,False
4,False,True,False,False,False
5,False,True,False,False,False
6,False,True,False,False,False
7,False,True,False,False,False
8,False,True,False,False,False
9,False,True,False,False,False


In [ ]:
#pass_data_df.set_index(["From_airport_IATA"]).count(level="From_airport_IATA")
pass_data_df.count(axis="From_airport_IATA")


In [ ]:
#pd.concat([obj_1_list_df, from_counts_df], keys=['Airport_IATA_code','Airport_IATA_code'])


In [ ]:
#pass_data_df['Depart_time_epoch'].dt.strftime("%m/%d/%y")
#pass_data_df['Depart_time_epoch']=
(pd.to_datetime(pass_data_df['Depart_time_epoch'],unit='s')) 


In [ ]:
obj1 = obj_1_list_df.merge(from_counts_df, how='outer', on=['Airport_IATA_code', 'Airport_IATA_code'])

In [28]:
if re.match(r"hello[0-9]+", 'hello1'):
    print('Yes')

Yes


In [64]:
pattern = r"[A-Z][A-Z][A-Z]"

pd.Series(
     ["DEN", "A;S", "AM%", "3b", "03c", "4dx"],
 ).str.contains(pattern)

0     True
1    False
2    False
3    False
4    False
5    False
dtype: bool

In [55]:
#pass_data_df[pass_data_df['From_airport_IATA'].str.match('DEN')== True]
#pass_data_df['From_airport_format']=
pass_data_df['From_airport_IATA'].str.match('[A-Z][A-Z][A-Z]')

0      True
1      True
2      True
3      True
4      True
       ... 
504    True
505    True
506    True
507    True
508    True
Name: From_airport_IATA, Length: 500, dtype: bool

In [170]:
#merging by IATA code
#select remaining from airports
from_values = flight_info_true.From_airport_IATA.unique()
dest_values = flight_info_true.Dest_airport_IATA.unique()
#turn to dataframe
from_values_df = pd.DataFrame(from_values)
dest_values_df = pd.DataFrame(dest_values)
#rename columns to allow merge
from_values_df.columns = ['Airport_IATA_code']
dest_values_df.columns = ['Airport_IATA_code']

In [192]:
from_merge['From_airport_exists']=from_merge['Airport_name'].str.contains(r"[A-Z]")
dest_merge['Dest_airport_exists']=dest_merge['Airport_name'].str.contains(r"[A-Z]")
#from_merge[from_merge['Airport_name'].str.contains('NaN')==False]
#new_df = df[df["col"].str.contains(word) == False]